# MapReduce

Tomado y adaptado del curso de Big Data por Raúl Ramos Pollán [GitHub](https://github.com/rramosp/20172.bigdata).

## Procesamiento de información con MAP-REDUCE

### Un ejemplo
Tenemos un log de la siguiente manera

    2012-01-01 09:08 BOG Libros 88.56 Discover
    2012-01-01 09:52 BGA Libros 62.41 Discover
    2012-01-01 10:08 MED Musica 93.37 Visa
    2012-01-01 10:58 MED Musica 395.93 Discover
    2012-01-01 14:38 BOG Musica 113.24 MasterCard
    2012-01-01 14:44 BGA Libros 290.5 Visa
    2012-01-01 16:26 MED Musica 246.12 Efectivo

Con el `dia`, `hora`, `ciudad`, `producto`, `importe` y `medio de pago` de compras realizadas en almacenes de una cierta cadena. Queremos obtener el total del importe de compras realizadas en cada ciudad. Una forma de hacerlo es la siguiente:

1) procesamos cada linea y, de cada una, generamos un par `(ciudad, importe)`, obteniendo los siguientes pares:

    (BOG, 88.56)
    (BGA, 62.41)
    (MED, 93.37)
    (MED, 395.93)
    (BOG, 113.24)
    (BGA, 290.5)
    (MED, 246.12)
    
2) agrupamos las tuplas generadas por el valor del primer componente:

    (BOG, [88.56, 113.24])
    (BGA, [62.41, 290.5])
    (MED, [93.37, 395.93, 246.12])
     
3) sumamos los elementos de cada lista para cada ciudad:

    (BOG, 201.8)
    (BGA, 352.91)
    (MED, 735.42)
    



### Tres fases

El primer paso en el ejemplo anterior se denomina **MAP** y, para cada registro de entrada, genera una tupla de formato `(clave, valor)`.

El segundo paso se denomina **SHUFFLE** y lo que hace es recopilar todas las tuplas generadas en el MAP anterior de cada `clave` y construir una lista con todos los valores generados. Es decir, una tupla de formato `(clave, lista_de_valores)` para cada clave.

El tercer paso se denomina **REDUCE** y, para cada clave, agrega los resultados de la lista generada en el SHUFFLE anterior.

Esta forma de procesar los datos constituye un **modelo de programación** llamado *MAP-REDUCE* y que está implementado por varios frameworks de programación y en varios lenguajes, de forma que el programador **solo implementa las funciones MAP y REDUCE** y el framework se encarga del shuffle.

### mr-job


Usaremos el framework [mr-job](https://pythonhosted.org/mrjob) para hacer nuestros programas map-reduce. El siguiente código implementa el proceso que acabamos de describir. Fíjate cómo sólo programamos las funciones `mapper` y `reducer`.

Si están trabajando en Google Colab pueden usar `!pip install mrjob` para instalar la librería.

In [ ]:
%%writefile py-files/mr-basico.py
from mrjob.job import MRJob
import os

class Compras(MRJob):

    def mapper(self, _, line):
        tokens = line.split()
        yield tokens[2], float(tokens[4])

    def reducer(self, key, values):
        yield key, sum(values)
        
if __name__ == '__main__':
    c = Compras()
    c.run()


In [ ]:
%%script python py-files/mr-basico.py -r inline --quiet
2012-01-01 09:08 BOG Libros 88.56 Discover
2012-01-01 09:52 BGA Libros 62.41 Discover
2012-01-01 10:08 MED Musica 93.37 Visa
2012-01-01 10:58 MED Musica 395.93 Discover
2012-01-01 14:38 BOG Musica 113.24 MasterCard
2012-01-01 14:44 BGA Libros 290.5 Visa
2012-01-01 16:26 MED Musica 246.12 Efectivo

Prueba a eliminar el término `--quiet` de la celda anterior y así verás los mensajes que emite **mr-job** durante la ejecución del programa. Útil para diagnosticar errores en la ejecución del código.

In [ ]:
%%script python py-files/mr-basico.py -r inline
2012-01-01 09:08 BOG Libros 88.56 Discover
2012-01-01 09:52 BGA Libros 62.41 Discover
2012-01-01 10:08 MED Musica 93.37 Visa
2012-01-01 10:58 MED Musica 395.93 Discover
2012-01-01 14:38 BOG Musica 113.24 MasterCard
2012-01-01 14:44 BGA Libros 290.5 Visa
2012-01-01 16:26 MED Musica 246.12 Efectivo

### Otro ejemplo

En este ejemplo partimos de una tuplas (personaA, personaB) que representa que la personaA considera a la personaB como amiga. La relación no es simétrica. El siguiente programa cuenta cuantas amigos considera cada persona que tiene.

Fíjate que la entrada es en formato JSON y cómo usamos la librería `json` de Python para convertir la entrada JSON en una lista de valores.

```
["juan", "pepe"]
["juan", "sebastian"]
["raul", "pepe"]
["ana", "pepe"]
["juan", "ana"]
["ana", "pedro"]
```

1) procesamos cada linea y, de cada una, generamos un par `(ciudad, importe)`, obteniendo los siguientes pares:

    (juan, 1)
    (juan, 1)
    (raul, 1)
    (ana, 1)
    (juan, 1)
    (ana, 1)
    
2) agrupamos las tuplas generadas por el valor del primer componente:

    (juan, [1, 1, 1])
    (raul, [1])
    (ana, [1, 1])
     
3) sumamos los elementos de cada lista para cada ciudad:

    (juan, 3)
    (raul, 1)
    (ana, 2)

In [ ]:
%%writefile py-files/mr-amigos.py
from mrjob.job import MRJob
import json

class Amigos(MRJob):

    def mapper(self, _, line):
        record = json.loads(line)
        yield record[0], 1

    def reducer(self, key, values):
        yield key, sum(values)
        
if __name__ == '__main__':
    c = Amigos()
    c.run()


In [ ]:
%%script python py-files/mr-amigos.py -r inline --quiet
["juan", "pepe"]
["juan", "sebastian"]
["raul", "pepe"]
["ana", "pepe"]
["juan", "ana"]
["ana", "pedro"]

## Instrumentación y runners

Usamos la instrumentación para encontrar errores, entender nuestro código, etc. Por ahora usamos `stderr.write` para saber cuantas veces se llama a cada función. Fíjate como el `reducer` se llama una vez por cada clave distinta que generamos en el `mapper`. Cuando usemos varios procesos o máquinas tendremos que recurrir a otros tipos de instrumentación.

Fíjate también que `values` en la función `reducer` es un **generador** [[ref](http://www.jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/)], es decir, no contiene en sí una lista de valores, sino que cada vez que va devolviendo valores uno a uno hasta que se acaban. Típicamente esto sucede porque va obteniendo los valores que devuelve de un _stream_ de entrada como un fichero, o una conexión remota. Por esto sólo podemos iterar sobre todos los valores **una única vez**.

Como ahora en el `reducer` queremos obtenter ambos el número de valores que tenemos y la suma de los mismos, tenemos que modificar nuestro código.

In [ ]:
%%writefile py-files/mr-basico-instrumentado.py
from mrjob.job import MRJob
from sys import stdin, stderr
import os

class Compras(MRJob):

    def mapper(self, _, line):
        tokens = line.split()
        stderr.write("MAPPER >> {0}\n".format(line))
        #print("mapper >>", line)
        yield tokens[2], float(tokens[4])
        
    def reducer(self, key, values):
        n_values, sum_values = 0, 0
        for i in values:
            n_values += 1
            sum_values += i
        stderr.write("REDUCER >> {0}, number of values {1}\n".format(key, n_values))
        #print("reducer >>", key, "number of values", n_values)
        yield key, sum_values
        
if __name__ == '__main__':
    c = Compras()
    c.run()

In [ ]:
%%script python py-files/mr-basico-instrumentado.py --quiet
2012-01-01 09:08 BOG Libros 88.56 Discover
2012-01-01 09:52 BGA Libros 62.41 Discover
2012-01-01 10:08 MED Musica 93.37 Visa
2012-01-01 10:58 MED Musica 395.93 Discover
2012-01-01 14:38 BOG Musica 113.24 MasterCard
2012-01-01 14:44 BGA Libros 290.5 Visa
2012-01-01 16:26 MED Musica 246.12 Efectivo

### Local runner

Los programas en **mr-job** pueden ejecutarse en distintos **runners**. Fíjate en la opción `-r` cuando ejecutamos nuestro código. Los runners posibles son los siguientes:

- **inline**: todos los `mapper` y `reducer` corren en el mismo proceso. Esta opción es útil para empezar a desarrollar un código y verificarlo funcionalmente.

- **local**: cada `mapper` y `reducer` corren en distintos procesos independientes en la misma máquina. Con esta opción podemos hacer una primera simulación de nuetro código en un entorno distribuido.

- **hadoop**: nuestro código se ejecuta en un cluster Hadoop

Instrumentar código con un local runner o en Hadoop ya no es tan fácil porque cada ejecución de las funciones `mapper` y `reducer` se hace en procesos o en máquinas distintas. Si no tenemos un mecanismo para recoger y coordinar la instrumentación generada en los distintos procesos o máquinas perderemos al menos parte de ella.


In [ ]:
%%script python py-files/mr-basico-instrumentado.py -r local --quiet
2012-01-01 09:08 BOG Libros 88.56 Discover
2012-01-01 09:52 BGA Libros 62.41 Discover
2012-01-01 10:08 MED Musica 93.37 Visa
2012-01-01 10:58 MED Musica 395.93 Discover
2012-01-01 14:38 BOG Musica 113.24 MasterCard
2012-01-01 14:44 BGA Libros 290.5 Visa
2012-01-01 16:26 MED Musica 246.12 Efectivo

Distintos frameworks usan distintos mecanismos para instrumentar el código distribuido. En este caso (mrjob/Hadoop) usamos los `counters`, y el framework asegura un estado global de los mismos de manera consistente. Ahora necesitamos mostrar los mensajes de salida del framework para ver el valor final de los contadores.

Usamos a partir de ahora el fichero `Data/compras.txt` que tiene 30 registros con el mismo formato que el usado hasta ahora.

In [ ]:
!cat ../Data/compras.txt

In [ ]:
%%writefile py-files/mr-basico-instrumentado-counters.py
from mrjob.job import MRJob
from sys import stdin
import os

class Compras(MRJob):

    def mapper(self, _, line):
        tokens = line.split()
        self.increment_counter("llamadas al map", "numero", 1)
        yield tokens[2], float(tokens[4])
        
    def reducer(self, key, values):
        n_values, sum_values = 0,0
        for i in values:
            n_values += 1
            sum_values += i
        self.increment_counter("longitud valores reduce", key, n_values )
        yield key, sum_values
        
if __name__ == '__main__':
    c = Compras()
    c.run()


In [ ]:
%%script python py-files/mr-basico-instrumentado-counters.py -r local ../Data/compras.txt
--

### Controlando el número de mappers y reducers
Ahora cambiamos la instrumentación para que cuente el número de llamadas de cada función por cada proceso. Puedes controlar el número de mappers y reducers al ejecutar tu programa como se indica abajo. Prueba a ejecutar el código con distintos números de mappers.

In [ ]:
%%writefile py-files/mr-basico-instrumentado-counters-tasks.py
from mrjob.job import MRJob
from sys import stdin
import os

class Compras(MRJob):

    def mapper(self, _, line):
        tokens = line.split()
        self.increment_counter("map: proceso "+str(os.getpid()), "numero", 1)
        yield tokens[2], float(tokens[4])
        
    def reducer(self, key, values):
        n_values, sum_values = 0,0
        for i in values:
            n_values += 1
            sum_values += i
        self.increment_counter("longitud valores reduce: proceso "+str(os.getpid()), key, n_values )
        yield key, sum_values
        
        
if __name__ == '__main__':
    c = Compras()
    c.run()

_la linea entera que se ejecuta a continuación es la siguiente_

    python files/mr-basico-instrumentado-counters-tasks.py 
           -r local 
           --jobconf mapred.map.tasks=5 
           --jobconf mapred.reduce.tasks=1 
           ../Data/compras.txt

In [ ]:
%%script python py-files/mr-basico-instrumentado-counters-tasks.py -r local --jobconf mapred.map.tasks=5 --jobconf mapred.reduce.tasks=1 ../Data/compras.txt
--

## Combiners

los combiners permiten resumir los datos que emite cada proceso map antes de que lleguen al reduce y así reducir el tráfico de red que sale de los procesos map y que entra en los reducers. Los combiners se ejecutan en la misma máquina que el map, alimentándose directamente de la salida de éste. Típicamente un combiner tiene la misma implementación que el `reducer` si es que éste **es asociativo**. Si no, puede tener otra implementación particular. Fíjate en la relación entre el número de procesos map y el número de valores que le entran a cada reduce.

In [ ]:
%%writefile py-files/mr-basico-combiners.py
from mrjob.job import MRJob
from sys import stdin
import os

class Compras(MRJob):

    def mapper(self, _, line):
        tokens = line.split()
        self.increment_counter("map: ", "numero", 1)
        yield tokens[2], float(tokens[4])
        
    def combiner(self, key, values):
        yield key, sum(values)
        
    def reducer(self, key, values):
        n_values, sum_values = 0,0
        for i in values:
            n_values += 1
            sum_values += i
        self.increment_counter("longitud valores reduce: ", key, n_values )
        yield key, sum_values
        
if __name__ == '__main__':
    c = Compras()
    c.run()

In [ ]:
%%script python py-files/mr-basico-combiners.py -r local ../Data/compras.txt
--

## Ejemplo WordCount

In [ ]:
%%writefile py-files/mr-basico-wordcount.py
from mrjob.job import MRJob
import re
WORD_RE = re.compile(r"[\w']+")

class MRWordFreqCount(MRJob):

    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            yield word.lower(), 1

    def combiner(self, word, counts):
        yield word, sum(counts)

    def reducer(self, word, counts):
        yield word, sum(counts)


if __name__ == '__main__':
    MRWordFreqCount.run()

In [ ]:
%%script python py-files/mr-basico-wordcount.py -r inline -q
Ancient influences have helped spawn variant interpretations 
of the nature of history which have evolved over the centuries 

In [ ]:
%%writefile py-files/mr-wordcount-max.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+")


class MRMostUsedWord(MRJob):

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        # sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]


if __name__ == '__main__':
    MRMostUsedWord.run()

In [ ]:
%%script python py-files/mr-wordcount-max.py -r inline -q ../Data/text.txt
--

In [ ]:
%%writefile py-files/mr-wordcount-max-clean.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+")


class MRMostUsedWord(MRJob):

    stopwords = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out',
    'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most',
    'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below',
    'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should',
    'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them',
    'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so',
    'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those',
    'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            if word.lower() not in self.stopwords:
                yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        # sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]


if __name__ == '__main__':
    MRMostUsedWord.run()

In [ ]:
%%script python py-files/mr-wordcount-max-clean.py -r inline -q ../Data/text.txt
--

## Hadoop

Para ejecutar la siguiente celda asegúrese de que los servicios de Hadoop están arriba y haya un archivo en la ruta `/user/<username>/grep/input`.

In [ ]:
# Reemplace <username> y <filename> según corresponda
%%script python py-files/mr-wordcount-max-clean.py -r hadoop --output-dir hdfs:///user/<username>/grep/output hdfs:///user/<username>/grep/input/<filename>
--